In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from tqdm.notebook import tqdm
import time
from selenium.webdriver.support import expected_conditions as EC

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)

C:\Users\mohit\AppData\Local\Temp\ipykernel_19108\472493040.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)
C:\Users\mohit\AppData\Local\Temp\ipykernel_19108\472493040.py:3: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = r'chromedriver.exe',chrome_options = chrome_options)


In [ ]:
inputlist = pd.read_excel(r"carefourinputlist.xlsx")
inputlist

In [ ]:
inputlist.drop_duplicates(inplace=True)
inputlist.dropna(inplace = True)
asinlist = list(inputlist['Asin '].unique())
asinlist

In [ ]:
df = pd.DataFrame(columns=['Asin','Category','Category Tree','Product URL','Product Title','Product Description','Product Features',
                            'Average Rating','Review Count','Ingredients','Product Benefits','Conservation',
                           'Operator & Consumer Service','Consumer Benefits','Price','Price Per Quanitity',
                           'Quantity','Allergens','Images URL'])

In [ ]:
for val in tqdm(range(0,len(asinlist))):
    
    i=asinlist[val]
    print(i)
    carefoururl = "https://www.carrefour.fr/p/-{}".format(i)
    driver.get(carefoururl)
    print(carefoururl)
    
    
    #click all the seemore available 
    #ingredient
    try:
        voir_plus_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[4]/div[1]/div/button")))
        voir_plus_button.click()
        time.sleep(2)
    except:
        pass
    #productbenefit
    try:    
        voir_plus_button = WebDriverWait(driver, 1).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[6]/div/div/button")))
        voir_plus_button.click()
        time.sleep(1)
    except:
        pass
    #consumerbenefit
    try:
        voir_plus_button = WebDriverWait(driver, 1).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[6]/div[2]/div/button")))
        voir_plus_button.click()
        time.sleep(1)
    except:
        pass
    
    #refreshing the page
    soup=BeautifulSoup(driver.page_source,"html.parser")
    productcomplete=soup.find_all("div",class_="pdp__product")
    # Initialize an empty DataFrame to store the extracted data
    try:
        aa=soup.find_all('li',class_="breadcrumb-trail__item")
        categorytree=[x.text.strip() for x in aa[:-1]]
        print(categorytree)
        
    except:
        categorytree=""
            
        
    #category
    try:
        cat1 = soup.find("div",class_="regulated-name")
        category=cat1.find("p",class_="pl-text pl-text--size-m pl-text--style-p").text
        print(category)
    except:
        category=""
    #Title
    try :
        prodtitle = soup.find('h1', class_='pl-text--style-subtitle').get_text(strip=True)
        print("Product Title:",prodtitle) 
    except:
        prodtitle =""

    #description
    try:
        product_description = soup.find('div', class_='pl-text--style-p').get_text(strip=True)
    # Print the product description
        print("Product Description:",product_description)
    except:
        product_description=""
        


    #nutritional values 
    try:
        nutritional_values_table = soup.find('table', class_='nutritional-details')
        nutritional_values_rows = nutritional_values_table.find_all('tr', class_='nutritional-fact')
        nutritional_values_dict = {}

        for row in nutritional_values_rows:
            name = row.find('th', class_='nutritional-fact__name').get_text(strip=True)
            value_per_quantity = row.find('td', class_='nutritional-fact__value nutritional-fact__center').get_text(strip=True)
            value_percent_element = row.find('td', class_='nutritional-fact__value nutritional-fact__right')
            value_percent = value_percent_element.get_text(strip=True) if value_percent_element else None

            nutritional_values_dict[name] = {
                "value_per_quantity": value_per_quantity,
                "value_percent": value_percent
            }

        # Print the fetched nutritional details
        print("Nutritional Values:")
        for name, values in nutritional_values_dict.items():
            if values["value_percent"]:
                print(f"{name}: {values['value_per_quantity']} ({values['value_percent']})")
            else:
                print(f"{name}: {values['value_per_quantity']}")
    except:
        nutritional_values_dict=""
   
         #product benefits 
    try:
#         voir_plus_button = WebDriverWait(driver, 5).until(
#         EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[6]/div/div/button")))
#         voir_plus_button.click()
    
#         # Wait for the content to update and parse the updated HTML using BeautifulSoup
#         updated_html = driver.page_source
#         soup = BeautifulSoup(updated_html, "html.parser")

        # Find the "Ingrédients" section
        benefit= soup.find("div", class_="product-benefits secondary-details__block")
        productb= benefit.find_all("div",class_="product-block-content")
        productbenefit=productb[0].get_text(strip=True)
        # Print the "Ingrédients" text
        print(productbenefit)
    except:
        productbenefit=""
 
        #consumer benefits
    try:
#         voir_plus_button = WebDriverWait(driver, 5).until(
#         EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[6]/div[2]/div/button")))
#         voir_plus_button.click()

#         # Wait for the content to update and parse the updated HTML using BeautifulSoup
#         updated_html1 = driver.page_source
#         soup1 = BeautifulSoup(updated_html1, "html.parser")

        # Find the "Ingrédients" section
        benefit= soup.find("div", class_="product-benefits secondary-details__block")
        consumerb= benefit.find_all("div",class_="product-block-content")
        consumerbenefit=consumerb[1].get_text(strip=True)
        # Print the "Ingrédients" text
        print(consumerbenefit)
    except:
        consumerbenefit=""

    #product allergens
    try:
        div_element = soup.find("div", {"data-testid": "product-composition-allergens", "class": "product-composition-allergens"})
        allergens = div_element.find("span", class_="pl-text pl-text--size-m pl-text--style-p").text
        print("Allergens:",allergens)
    except:
        allergens =""
    
    #price 
    finalprice = ""  # Initialize the finalprice variable as an empty string

    try:
        price_element = soup.find("div", {"data-testid": "product-card-price", "class": "product-card-price"})
        price = price_element.find("span", {"data-testid": "price-displayed", "class": "pl-text pl-text--size-m pl-text--style-p pl-text--bold product-card-price--final"}).text
        finalprice = price  # If price is available, assign it to finalprice
    except AttributeError:  # Handle the AttributeError in case the price is not available
        try:
            price_element2 = soup.find("div", {"data-testid": "product-card-price", "class": "product-card-price"})
            discountedprice = price_element2.find("span", {"data-testid": "price-displayed", "class": "pl-text pl-text--size-m pl-text--style-p pl-text--bold product-card-price--final product-card-price__discount--promo"}).text
            finalprice = discountedprice  # If discounted price is available, assign it to finalprice
        except AttributeError:  # Handle the AttributeError in case the discounted price is also not available
            try:
                price_element3 = soup.find("div", {"data-testid": "product-card-price", "class": "product-card-price"})
                discountedprice1 = price_element3.find("span", {"class": "pl-text pl-text--size-m pl-text--style-p pl-text--bold product-card-price--final product-card-price__discount--fid"}).text
                finalprice = discountedprice1  # If discounted price is available, assign it to finalprice
            except AttributeError:
                finalprice = "" 
    print(finalprice)
    

    #price_per_quantity
    try:
        price_per = soup.find("div",class_="pdp-pricing__group")
        priceperquantity = price_per.find("p",class_="pl-text pl-text--size-s pl-text--style-p").text
    
    except:
        priceperquantity =""
    
    #quanitity
    try:
        quantity_element = soup.find("div",class_="main-details__header mobile-only")
        quantity = quantity_element.find("span",class_="pl-text pl-text--size-m pl-text--style-p pl-text--bold").text
    
    except:
        quantity =""
    
    #image data
    #first block is for single image
    try:
        image_element1 = soup.find("div", class_="single-image__wrapper")
        zoomable_image_div = image_element1.find("div", class_="zoomable-image")
        image_element = zoomable_image_div.find("img", class_="zoomable-image__image")
        image_url = image_element["data-src"]
        print(image_url)
    except: #second block is for multiple image
        try:
            image_elements = soup.find_all('img', class_="image ds-visual-picker__image")
            product_images = [img['src'] for img in image_elements]
            product_images1 = [x.replace("p_43x43", "p_540x540") for x in product_images]
            print("Product Images:")
            for image_url in product_images1:
                print(image_url)
        except:
            image_url = ""
            print("Product Image not available.")

    #rating&review Summary
    try:
         # Find the rating summary element and extract the text
        rating_summary_element = soup.find('p', class_='rating-summary__average')
        rating_summary = rating_summary_element.get_text(strip=True)

        # Find the review count element and extract the text
        review_count_element = soup.find('p', class_='rating-summary__count')
        review_count = review_count_element.get_text(strip=True)
        # Print the rating summary and review count
        print("Average Rating:", rating_summary)
        print("Review Count:", review_count)
    except:
        rating_summary=""
        review_count=""
        
    
    #Product Features
    try:
        product_features_heading = soup.find("div",class_="product-badge-anchor-heading").text
        content_features = soup.find("div",class_="product-badge-anchor-content").text
        # Format and print the output
        product_features = f"{product_features_heading} : {content_features}"
    
    except:
        product_features = ""
        
   
        
    #Advice & Consumer Info
    try:
        advice_consumer_info = soup.find("div",class_="product-informations secondary-details__block")
        advice_consumer_info1 = advice_consumer_info.find_all("span",class_="pl-text pl-text--size-m pl-text--style-p")
        conservation=advice_consumer_info1[0].text
        Operator_contact_Consumer_service=advice_consumer_info1[1].text
        
    except:
        conservation=""
        Operator_contact_Consumer_service=""
        
      
    #Ingredient Complete
#     try:
#         while True:
#             voir_plus_button = WebDriverWait(driver,5).until(
#             EC.element_to_be_clickable((By.XPATH, "/html/body/main/div/div/div/div[2]/div/div[1]/div[4]/div[1]/div/button")))
#             voir_plus_button.click()
            
#     except:
#         pass
#         # Wait for the content to update and parse the updated HTML using BeautifulSoup
#     updated_html = driver.page_source
#     soup1 = BeautifulSoup(updated_html, "html.parser")
    
    try:
        # Find the "Ingrédients" section
        ingredient1= soup.find("div", {"data-testid": "product-composition-default-content", "class": "product-block-content"})
        ingredient= ingredient1.find("span", class_="pl-text pl-text--size-m pl-text--style-p").text
        # Print the "Ingrédients" text
        print(ingredient)
    except:
        ingredient=""
    
    # Append the extracted information as a new row to the DataFrame
    data = {'Asin':i,'Product URL': carefoururl,'Category': category,'Category Tree': categorytree,
            'Nutritional Value':nutritional_values_dict,
            'Product Title': prodtitle,
            'Product Description': product_description,
            'Average Rating': rating_summary,
            'Review Count': review_count,
            'Ingredients': ingredient,
            'Product Features': product_features,
            'Product Benefits': productbenefit,
            'Consumer Benefits': consumerbenefit,
            'Price': finalprice,
            'Price Per Quanitity': priceperquantity,
            'Quantity': quantity,
            'Allergens': allergens,
            'Images URL': product_images1,
            'Conservation': conservation,'Operator & Consumer Service': Operator_contact_Consumer_service}
    df = df.append(data, ignore_index=True)
    time.sleep(1)

# Print the extracted shop details
print(df)

In [ ]:
df.head()

In [ ]:
df.to_excel(r'finaloutput_carefour_content_score_ALL.xlsx')